# Lab 3: Iterative Algorithm Design
## Dominic Zagra, Evan Schubert, Zach Wentz


### Worst Case Run Time Analysis
For this algorithm, our "n" (the input that can change in size) is the number of vertices of a shape. 

In [2]:
def point_inside(vertices, classification_point):
    """Determines if the classification point is in the polygon.
        
        Parameters:
        vertices (Tuple): Vertices of the polygon.
        classification_point (Tuple): 2D point to determine if it's in the polygon.
        
        Returns:
        ret_bool (bool): Whether the classification point is in the polygon or not."""
    num_hits = 0
    classification_x, classification_y = classification_point

    for i in range(len(vertices)):
        vertex_X1, vertex_Y1 = vertices[i]
        vertex_X2, vertex_Y2 = vertices[(i+1) % len(vertices)]

        #what if y's on the vertices are equal?
        #what happens if the classifiction point is on the line of the polygon, answer: it is out of the polygon
        if (classification_y > min(vertex_Y1, vertex_Y2) and classification_y <= max(vertex_Y1, vertex_Y2)):  
            if (vertex_X1 != vertex_X2):
                slope = (vertex_Y2 - vertex_Y1) / (vertex_X2 - vertex_X1)
                intercept = vertex_Y2 - (slope * vertex_X2)
                
                on_line_x = (classification_y - intercept) / slope

            else:
                on_line_x = vertex_X2
                
            if (on_line_x > classification_x):
                    num_hits += 1
            
    print(f"Number of hits: {num_hits}")
    if (num_hits % 2 == 0):
        return False
    else:
        return True

In [3]:
def test_algo():
    # testing for a square, with point outside of it
    square_vertices = ((1,1), (1,2), (2,2), (2,1))
    classification_point = ((2,3))
    assert point_inside(square_vertices, classification_point) == False

    # testing for a square, with point inside of it
    classification_point_2 = ((1.5, 1.5))
    assert point_inside(square_vertices, classification_point_2) == True
    
    #testing for a triangle, with point outside of it
    triangle_vertices = ((1,1), (1.5,2), (2,1))
    classification_point_3 = ((1,2))
    assert point_inside(triangle_vertices, classification_point_3) == False

    #testing for a triangle, with point inside of it
    classification_point_4 = ((1.5, 1.5))
    assert point_inside(triangle_vertices, classification_point_4) == True

    #testing for a triangle, with a point (0,1)
    classification_point_5 = ((0,1))
    assert point_inside(triangle_vertices, classification_point_5) == False

    #test
    triangle_vertices = ((1, 1), (1, 3), (2, 2))
    classification_point_6 = ((1.5, 2))
    assert point_inside(triangle_vertices, classification_point_6) == True

    #test point outside upside down pentagon (horizontal line hits lower tip of pentagon)
    pentagon_vertices = ((3, 0), (2, 1), (2, 3), (4, 3), (4, 1))
    classification_point_7 = ((0, 0))
    assert point_inside(pentagon_vertices, classification_point_7) == False


    shape_vertices = ((1, 1), (1, 4), (6, 4), (6, 3), (4, 3), (2, 1))
    classification_point_8 = ((3, 3))
    assert point_inside(shape_vertices, classification_point_8) == True


In [4]:
test_algo()

Number of hits: 0
Number of hits: 1
Number of hits: 2
Number of hits: 1
Number of hits: 0
Number of hits: 1
Number of hits: 0
Number of hits: 1


## Test Cases


| Test Case                               | Expected Answer     | Algo Returned Answer |
| ------------------------------------------------ | ------------------- | -------------------- |
| <img src="TestCase1.png" width="250" height="200">            | False (Point not in polygon)| False (Point not in polygon)                |
| <img src="TestCase2.png" width="250" height="200">            | True (Point in polygon)| True (Point in polygon)                |
| <img src="TestCase3.png" width="250" height="200">            | False (Point not in polygon)| False (Point not in polygon)                |
| <img src="TestCase4.png" width="250" height="200">            | True (Point in polygon)| True (Point in polygon)                |
| <img src="TestCase5.png" width="250" height="200">            | False (Point not in polygon)| False (Point not in polygon)                |
| <img src="TestCase6.png" width="250">            | True (Point in polygon)| True (Point in polygon)                |
| <img src="TestCase7.png" width="250" height="200">            | False (Point not in polygon)| False (Point not in polygon)                |
| <img src="TestCase8.png" width="250" height="200">            | True (Point in polygon)| True (Point in polygon)                |

